# Question 1:  Pose Graph Optimization for 1D SLAM

A solved example for 1D SLAM which optimizes for pose variables using weighted least squares method (Gauss Newton) has been explained in the class. You can view it at `examples/1D-SLAM.ipynb`. Your first task is to code this from scratch.

For this section, you have to calculate Jacobian analytically yourself and use it. However, you can check how correct it is using `jax`'s `jacobian`. Its usage is explained in the supplementary notebook (`examples/`).

## How to get started?

1. Go through the example and understand the optimization procedure.
2. Generate the data as in the example.
3. Write the derivatives of the function with respect to the constraints
4. Create the final jacobian and write the update steps using Gauss Newton

This tiny example will help make sense of the next question, as it's largely just an extension of this.

Finally, plot the loop closure constraint, poses, and error at every iteration as well. Since this is a one dimensional example, you can just assume x = y.

In [2]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

In [47]:
class Optimizer:
    def __init__(self, f: Callable[[np.ndarray], np.ndarray],
                 jac: Callable[[np.ndarray], np.ndarray],
                 initial_values: np.ndarray, info_mat: np.ndarray, n_iter=10):
        """
        Create Gauss - Newton based optmizer for given loss_function and jacobial function
        :param loss_function:  Loss function
        :param jac: Function to compute jac at a state
        :param initial_values: Initial values of state
        """

        self.f = f
        self.jac = jac
        self.initial_values = initial_values
        self.info_mat = info_mat
        self.n_iter = n_iter

        self.F = lambda x: 0.5 * (f(x).T @ info_mat @ f(x))
        self.__current = self.initial_values.copy()
        self.__iter = 0

    def get_current(self):
        return self.__current
    def get_change(self) -> np.ndarray:
        J = self.jac(self.__current)
        H = J.T @ self.info_mat @ J
        b = J.T @ self.info_mat.T @ self.f(self.__current)
        dx = -np.linalg.inv(H) @ b

        return dx

    def print_info(self):
        print(f"Iteration: {self.__iter} / {self.n_iter}")
        print(f"Error: {self.F(self.__current)}")
        # TODO: Draw plots

    def update(self):
        self.__iter+=1
        dx = self.get_change()
        self.__current += dx

    def optimize(self):
        self.print_info()
        for i in range(self.n_iter):
            self.update()
            self.print_info()

In [23]:
initial_values = np.array([0, 1.1, 2.1, 3.2, 0.5]).reshape(-1, 1)
f = lambda x: np.array(
    [x[0] + 1.1 - x[1], x[1] + 1 - x[2], x[2] + 1.1 - x[3], x[3] - 2.7 - x[4],
     x[0] + 0 - x[4], x[0] - 0]).reshape(-1, 1)
jac = lambda x: np.array(
    [[1, -1, 0, 0, 0], [0, 1, -1, 0, 0], [0, 0, 1, -1, 0], [0, 0, 0, 1, -1],
     [1, 0, 0, 0, -1], [1, 0, 0, 0, 0]])
info_mat = np.eye(6)
info_mat *= 100
info_mat[5, 5] = 1000

In [15]:
info_mat

array([[ 100.,    0.,    0.,    0.,    0.,    0.],
       [   0.,  100.,    0.,    0.,    0.,    0.],
       [   0.,    0.,  100.,    0.,    0.,    0.],
       [   0.,    0.,    0.,  100.,    0.,    0.],
       [   0.,    0.,    0.,    0.,  100.,    0.],
       [   0.,    0.,    0.,    0.,    0., 1000.]])

In [48]:
solver = Optimizer(f, jac, initial_values, info_mat, 1)
# n_iter only 1 because its linear


In [49]:
solver.optimize()

Iteration: 0 / 1
Error: [[12.5]]
Iteration: 1 / 1
Error: [[2.5]]


In [44]:
solver.get_current()

array([[6.9388939e-18],
       [1.0000000e+00],
       [1.9000000e+00],
       [2.9000000e+00],
       [1.0000000e-01]])

In [46]:
solver.print_info()

Iteration: 0 / 1
Error: [[12.5]]
